In [5]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.circuit import Parameter

In [6]:
def create_qaoa_circ(graph, theta):
    """Creates a parametrized qaoa circuit
    Args:
        graph: networkx graph
        theta: (list) unitary parameters
    Returns:
        (QuantumCircuit) qiskit circuit
    """
    nqubits = len(graph.nodes())
    n_layers = len(theta)//2  # number of alternating unitaries
    beta = theta[:n_layers]
    gamma = theta[n_layers:]

    qc = QuantumCircuit(nqubits)

    # initial_state
    qc.h(range(nqubits))

    for layer_index in range(n_layers):
        # problem unitary
        for pair in list(graph.edges()):
            qc.rzz(2 * gamma[layer_index], pair[0], pair[1])
        # mixer unitary
        for qubit in range(nqubits):
            qc.rx(2 * beta[layer_index], qubit)

    qc.measure_all()
    return qc

In [12]:
edges = []
for j in range(9):
    for i in range(9):
        if i < j:
            edges.append((i,j))
#print(edges)

weights = [606.5,606.5,606.5,606.5,15.75,15.75,15.75,606.5,15.75,606.5,15.75,15.75,606.5,606.5,606.5,606.5,22.75,22.75,606.5,12,12,22.75,606.5,22.75,12,606.5,12,606.5,22.75,22.75,606.5,12,12,606.5,606.5,606.5]

graph = dict(zip(edges,weights))
print(graph) # all the zz terms and their weights

{(0, 1): 606.5, (0, 2): 606.5, (1, 2): 606.5, (0, 3): 606.5, (1, 3): 15.75, (2, 3): 15.75, (0, 4): 15.75, (1, 4): 606.5, (2, 4): 15.75, (3, 4): 606.5, (0, 5): 15.75, (1, 5): 15.75, (2, 5): 606.5, (3, 5): 606.5, (4, 5): 606.5, (0, 6): 606.5, (1, 6): 22.75, (2, 6): 22.75, (3, 6): 606.5, (4, 6): 12, (5, 6): 12, (0, 7): 22.75, (1, 7): 606.5, (2, 7): 22.75, (3, 7): 12, (4, 7): 606.5, (5, 7): 12, (6, 7): 606.5, (0, 8): 22.75, (1, 8): 22.75, (2, 8): 606.5, (3, 8): 12, (4, 8): 12, (5, 8): 606.5, (6, 8): 606.5, (7, 8): 606.5}


In [13]:
W = []
for i in range(len(edges)):
    W.append((edges[i][0],edges[i][1],weights[i]))
print(W)

[(0, 1, 606.5), (0, 2, 606.5), (1, 2, 606.5), (0, 3, 606.5), (1, 3, 15.75), (2, 3, 15.75), (0, 4, 15.75), (1, 4, 606.5), (2, 4, 15.75), (3, 4, 606.5), (0, 5, 15.75), (1, 5, 15.75), (2, 5, 606.5), (3, 5, 606.5), (4, 5, 606.5), (0, 6, 606.5), (1, 6, 22.75), (2, 6, 22.75), (3, 6, 606.5), (4, 6, 12), (5, 6, 12), (0, 7, 22.75), (1, 7, 606.5), (2, 7, 22.75), (3, 7, 12), (4, 7, 606.5), (5, 7, 12), (6, 7, 606.5), (0, 8, 22.75), (1, 8, 22.75), (2, 8, 606.5), (3, 8, 12), (4, 8, 12), (5, 8, 606.5), (6, 8, 606.5), (7, 8, 606.5)]


In [11]:
lattice_cite = [i for i in range(9)]
field_str = [-1290,-1290,-1290,-1268.5,-1268.5,-1268.5,-1282.5,-1282.5,-1282.5]
z_terms = dict(zip(lattice_cite,field_str))
print(z_terms) # all the z terms and their weights

{0: -1290, 1: -1290, 2: -1290, 3: -1268.5, 4: -1268.5, 5: -1268.5, 6: -1282.5, 7: -1282.5, 8: -1282.5}


In [14]:
def create_qaoa_circ_modified(W, theta, field_str):
    """Creates a parametrized qaoa circuit
    Args:
        W: (tuple) containing edges and its respective weights 
        theta: (list) unitary parameters
        
    Returns:
        (QuantumCircuit) qiskit circuit
    """
    nqubits = len(G.nodes())
    n_layers = len(theta)//2  # number of alternating unitaries
    beta = theta[:n_layers]
    gamma = theta[n_layers:]

    qc = QuantumCircuit(nqubits)

    # initial_state
    qc.h(range(nqubits))

    for layer_index in range(n_layers):
        #problem unitary
        for i in range(nqubits):
            qc.rz(2*gamma[layer_index]*field_str[i],i )
        for pair in W:  # pairs of nodes
            qc.rzz(2 * gamma[layer_index]*pair[2], pair[0], pair[1])
        # mixer unitary
        for qubit in range(nqubits):
            qc.rx(2 * beta[layer_index], qubit)

    qc.measure_all()
    return qc

In [7]:
#nv = number of vertices , cf = cost fun
# x = parameters
param = [1,1]
G = nx.Graph()
G.add_nodes_from([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
G.add_edges_from([(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 6), (5, 7), (5, 8), (5, 9), (6, 7), (6, 8), (6, 9), (7, 8), (7, 9), (8, 9)])

def Energy(cf, nv, x):
    Edges = [(i, j) for i, j in G]
    #qc_res = create_qaoa_circ(cf, nv, x)
    qc = create_qaoa_circ(G,param)
    backend = Aer.get_backend('statevector_simulator')
    job = execute(qc, backend)
    psi = job.result().get_statevector(qc, decimals=16)
    Exp_value = []
    for edges in G.edges():
        qc_res1 = create_qaoa_circ(G,param)
        for vertices in edges: # building the cost hamiltonian 
            qc_res1.z(vertices) # this is like adding the zz operator to each edge
            job1 = execute(qc_res1, backend)
            phi = job1.result().get_statevector(qc_res1, decimals=16)
            Exp_value.append(np.dot(np.transpose(np.conjugate(psizz)), phi))
    return np.real(np.sum(Exp_value))


In [ ]:
# applying a cost hamiltonian H_c of TSP to QAOA anstaz 

def state_phi_zz(): # appending sll the zz terms 
    for edges in graph.edges():
        psi_zz = create_qaoa_circ_modified(W, param, field_str)
        for vertices in edges: # building the cost hamiltonian
            psi_zz.z(vertices) # this is like adding the zz operator to each edge
            job1 = execute(psi_zz, backend)
            phi = job1.result().get_statevector(psi_zz, decimals=16)
            Exp_value.append(np.dot(np.transpose(np.conjugate(psizz)), phi))
    return np.real(np.sum(Exp_value))



